In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4'


In [2]:
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import itertools
import pickle
from tqdm import tqdm
import csv
import re
from transformers import AutoTokenizer, LlamaForCausalLM
from tqdm import tqdm
import torch

ModuleNotFoundError: No module named 'torch'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42) # Seed 고정

In [4]:
model_path = "Phind/Phind-CodeLlama-34B-v2"

In [5]:
# initialize the model


model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████| 7/7 [17:01<00:00, 145.99s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at Phind/Phind-CodeLlama-34B-v2 and are newly initialized: ['model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.47.self_attn.rotary_emb.inv_freq', 'model.layers.36.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.43.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.39.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.38.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rota

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [11]:
print(tokenizer(inputtexts, return_tensors="pt", truncation=True, max_length=4096).input_ids.to("cuda").squeeze())

tensor([    1, 29871,    13,  2277, 29937,  2184,  9705,   415,    13,  3492,
          526,   385, 13052,   296,  8720, 20255, 29889,    13,    13,  2277,
        29937,  4911,  7777,    13,  1888,  2037,   263,  9024,  1051,   297,
          315,  1817,    13,    13,  2277, 29937,  4007, 22137,    13,   856,
           13], device='cuda:0')


In [19]:




# HumanEval helper

def generate_one_completion(prompt: str):
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)
    # Generate
    generate_ids = model.generate(inputs.input_ids.to("cuda"), max_new_tokens=384, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
    completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    completion = completion.replace(prompt, "").split("\n\n\n")[0]

    return completion

# perform HumanEval
inputtexts = """
### System Prompt
You are an intelligent programming assistant.

### User Message
Implement a linked list in C++

### Assistant
...
"""

print(generate_one_completion(inputtexts))

RuntimeError: shape '[1, 41, 64, 128]' is invalid for input of size 41984

In [ ]:
MODEL_NAME = "Phind/Phind-CodeLlama-34B-v2"
MODEL_TAG = "Phind/Phind-CodeLlama-34B-v2"
root_dir = "/home/leadawon5/decs_jupyter_lab/gitfiles/DACON/code_similarity/bigdata/train_code" 






In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# tokenizer와 model은 미리 정의되어 있어야 합니다.
# device는 'cuda' 또는 'cpu'일 수 있습니다.

def predict(model, tokenizer, test_data, device, threshold=0.5):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
            # 코드 쌍을 토큰화합니다.
            # inputs1 = tokenizer(row['code1'], return_tensors='pt', max_length=512, padding='max_length', truncation=True).to(device)
            # inputs2 = tokenizer(row['code2'], return_tensors='pt', max_length=512, padding='max_length', truncation=True).to(device)
            
            assert False
            
            # 모델을 통해 유사도 점수(로짓)를 계산합니다.
            logits = model(inputs1['input_ids'], inputs1['attention_mask'], inputs2['input_ids'], inputs2['attention_mask'])
            
            # 로짓을 확률로 변환하기 위해 sigmoid 함수를 적용합니다.
            probs = torch.sigmoid(logits).cpu().numpy()
            
            # 설정한 임계값을 기준으로 유사 여부를 판단합니다.
            prediction = 1 if probs > threshold else 0
            predictions.append(prediction)
    
    return predictions

# 예제 사용
test_data = pd.read_csv("./bigdata/test.csv")
# 모델과 tokenizer가 정의되어 있어야 합니다.
predictions = predict(model, tokenizer, test_data, device, threshold=0.5)

# 결과를 제출 파일로 저장
submission = pd.read_csv('./bigdata/sample_submission.csv')
submission['similar'] = predictions
submission.to_csv('./bigdata/predictions_submit.csv', index=False)